In [1]:
import duckdb as dd

# Create an in-memory DuckDB connection
con = dd.connect(':memory:')

In [2]:
con = dd.connect('my_database.db')
con.sql('SHOW ALL TABLES')

┌──────────┬─────────┬─────────┬──────────────┬──────────────┬───────────┐
│ database │ schema  │  name   │ column_names │ column_types │ temporary │
│ varchar  │ varchar │ varchar │  varchar[]   │  varchar[]   │  boolean  │
├──────────┴─────────┴─────────┴──────────────┴──────────────┴───────────┤
│                                 0 rows                                 │
└────────────────────────────────────────────────────────────────────────┘

In [3]:
#con.sql('DROP TABLE IF EXISTS fuels;')

In [5]:
con.execute('''
    CREATE TABLE brands (
        brandname TEXT unique
    );
''')

In [6]:
con.execute('''
    CREATE TABLE fuels (
        fuelcode TEXT unique
    );
''')

In [7]:
con.execute('''
    CREATE TABLE stations (
        stationname TEXT unique,
        brand TEXT REFERENCES brands(brandname),
        address TEXT,
        suburb TEXT,
        postalcode TEXT,
        lat DOUBLE,
        lon DOUBLE,
    );
''')

In [8]:
con.execute('''
    CREATE TABLE prices (
        fuelcode TEXT REFERENCES fuels(fuelcode),
        station TEXT REFERENCES stations(stationname),
        price DOUBLE,
        updatetime TIMESTAMP
    );
''')

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'brandname': ['seven eleven'],
})

con.sql('''
        INSERT INTO brands (brandname)
        SELECT * FROM df
''')

In [ ]:
con.sql('select * from brands')

┌──────────────┐
│  brandname   │
│   varchar    │
├──────────────┤
│ seven eleven │
└──────────────┘

In [ ]:
df = pd.DataFrame({
    'fuelcode': ['94'],
})

con.sql('''
        INSERT INTO fuels (fuelcode)
        SELECT * FROM df
''')

In [ ]:
con.sql('select * from fuels')

CatalogException: Catalog Error: Table with name fuels does not exist!
Did you mean "pg_class"?

In [ ]:
df = pd.DataFrame({
    'stationname': ['station 1'],
    'brand': ['seven eleven'],
    'address': ['1 brisbane st'],
    'suburb': ['Chifley'],
    'postalcode': ['2036'],
    'lat': [50.300],
    'lon': [20.901],
})

con.sql('''
        INSERT INTO stations (stationname, brand, address, suburb, postalcode, lat, lon)
        SELECT * FROM df
''')

con.sql('select * from stations')

┌─────────────┬──────────────┬───────────────┬─────────┬────────────┬────────┬────────┐
│ stationname │    brand     │    address    │ suburb  │ postalcode │  lat   │  lon   │
│   varchar   │   varchar    │    varchar    │ varchar │  varchar   │ double │ double │
├─────────────┼──────────────┼───────────────┼─────────┼────────────┼────────┼────────┤
│ station 1   │ seven eleven │ 1 brisbane st │ Chifley │ 2036       │   50.3 │ 20.901 │
└─────────────┴──────────────┴───────────────┴─────────┴────────────┴────────┴────────┘

In [ ]:
from datetime import datetime

df = pd.DataFrame({
    'station': ['station 1'],
    'fuelcode': ['94'],
    'updatetime': [datetime.now()],
    'price': [210.5],
})

con.sql('''
        INSERT INTO prices (station, fuelcode, updatetime, price)
        SELECT * FROM df
''')

con.sql('select * from prices')

┌──────────┬───────────┬────────┬────────────────────────────┐
│ fuelcode │  station  │ price  │         updatetime         │
│ varchar  │  varchar  │ double │         timestamp          │
├──────────┼───────────┼────────┼────────────────────────────┤
│ 94       │ station 1 │  210.5 │ 2025-04-11 08:36:08.303049 │
│ 94       │ station 1 │  210.5 │ 2025-04-11 08:36:43.802418 │
└──────────┴───────────┴────────┴────────────────────────────┘

In [ ]:
import duckdb
import graphviz


tables = con.sql("""
    SHOW ALL TABLES;
""").fetchall()

tables

[('my_database', 'main', 'brands', ['brandname'], ['VARCHAR'], False),
 ('my_database', 'main', 'fuels', ['fuelcode'], ['VARCHAR'], False),
 ('my_database',
  'main',
  'prices',
  ['fuelcode', 'station', 'price', 'updatetime'],
  ['VARCHAR', 'VARCHAR', 'DOUBLE', 'TIMESTAMP'],
  False),
 ('my_database',
  'main',
  'stations',
  ['stationname', 'brand', 'address', 'suburb', 'postalcode', 'lat', 'lon'],
  ['VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'VARCHAR', 'DOUBLE', 'DOUBLE'],
  False)]